In [3]:
import os

os.environ["TF_ENABLE_ONEDNN_OPTS"] = "0"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

from tensorflow import keras

2025-01-30 12:37:29.861200: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738240650.018094   19751 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738240650.057306   19751 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()
(X_train, y_train), (X_test, y_test) = (X_train[:10000], y_train[:10000]), (X_test[:10000], y_test[:10000])

print((X_train.shape, y_train.shape), (X_test.shape, y_test.shape))

y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)

print((y_train.shape, y_test.shape))

((10000, 28, 28), (10000,)) ((10000, 28, 28), (10000,))
((10000, 10), (10000, 10))


In [5]:
X_train, X_test = X_train / 255.0, X_test / 255.0
X_train, X_test = X_train.reshape(X_train.shape[0], 1, X_train.shape[1], X_train.shape[2]), X_test.reshape(X_test.shape[0], 1, X_test.shape[1], X_test.shape[2])
print(X_train.shape, X_test.shape)


(10000, 1, 28, 28) (10000, 1, 28, 28)


In [6]:
from nea.ml.nn import Module, Conv2D, Dense, ReLU, Reshape, SGD, MSE, Softmax
from nea.ml.autograd import Tensor

class MNISTER(Module):
    def __init__(self):
        self.conv2d_1 = Conv2D(X_train.shape[1:], 3, 5)
        self.relu_1 = ReLU()
        self.conv2d_2 = Conv2D((5, 26, 26), 3, 1)
        self.relu_2 = ReLU()
        self.reshape_1 = Reshape((576, ))
        self.dense_1 = Dense(576, 256)
        self.relu_3 = ReLU()
        self.dense_2 = Dense(256, 10)
        self.softmax = Softmax()

    
    def forward(self, x_sample: Tensor) -> Tensor:
        out = self.conv2d_1(x_sample)
        out = self.relu_1(out)
        out = self.conv2d_2(out)
        out = self.relu_2(out)
        out = self.reshape_1(out)
        out = self.dense_1(out)
        out = self.relu_3(out)
        out = self.dense_2(out)
        out = self.softmax(out)
        return out

    def __call__(self, x_sample: Tensor) -> Tensor:
        return self.forward(x_sample)

In [8]:
from tqdm import tqdm
import gc

model = MNISTER()
optim = SGD(model.params)
loss_func = MSE()

epochs = 10
batch_size = 64

for epoch in range(epochs):
    print(f"---------EPOCH: {epoch}------------")

    for start_idx in tqdm(range(0, X_train.shape[0], batch_size)):
        end_idx = min(X_train.shape[0], start_idx + batch_size)
        
        loss = 0
        for sample in range(start_idx, end_idx):
            pred = model(X_train[sample])
            loss += loss_func(pred, y_train[sample])

    
    loss = loss.mean()
    optim.zero_grad()
    loss.backward()
    optim.step()

    gc.collect()

    


print("DONE")



---------EPOCH: 0------------


  0%|          | 0/157 [00:00<?, ?it/s]


AssertionError: <code object forward at 0x1c83f0e0, file "/mnt/c/Users/edgar/OneDrive/Documents/GitHub/NEA/nea/ml/autograd/tensor.py", line 972> != <code object cpu_forward_convolve2d at 0x7f92541a8f50, file "/mnt/c/Users/edgar/OneDrive/Documents/GitHub/NEA/nea/ml/autograd/convolve_funcs.py", line 6>